In [1]:
import numpy as py
import random as random
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
from scipy.stats import t
import math
import time
import imageio
import os


from Coord import *
from Submarine import *
from Merchant_Ship import Merchant_Ship

In [2]:
t = py.linspace(0,100, num = 1000)

In [3]:
def generate_objects(n_merch,n_tgts,n_subs,speed_sub):
    '''Generates a requested number of merchants, targets, and submarines. Submarine speed may be specified.'''
    
    # Lambda for interarrival 
    ld = 1000
    
    # Target name builder
    tgt_name = 'Target_'
    tgt_names = []
    for i in range(1,n_tgts+1):
        tgt_names.append(tgt_name + str(i))
        
    # Target builder
    time_delay = 0
    Targets = []
    for i in tgt_names:
        Targets.append(Merchant_Ship(i, Coord(random.uniform(0,100),0),time_delay))
        time_delay += py.random.exponential(ld)
        
        
    # Merchant name builder
    merch_name = 'Merchant_'
    merch_names = []
    for i in range(1,n_merch+1):
        merch_names.append(merch_name + str(i))
        
    # Merchant builder
    time_delay = 0
    Merchants = []
    for j in merch_names:
        Merchants.append(Merchant_Ship(j, Coord(random.uniform(0,100),0),time_delay))
        time_delay += py.random.exponential(ld)

    # Submarine name builder
    sub_name = 'Hunter_'
    sub_names = []
    for i in range(1,n_subs+1):
        sub_names.append(sub_name + str(i))
        
    # Submarine builder
    Submarines = []
    for i in sub_names:
        location = Coord(random.uniform(0,100),100)
        course = round(random.random())*180
        Submarines.append(Submarine(location,crs = course, spd = speed_sub))
        
    return Targets,Merchants,Submarines


In [4]:
def contact_picture(tgt_list,merch_list,sub_list,torp_list):
    '''Plots contact picture'''
    
    # Targets 
    for item_t in tgt_list:
        plt.plot(item_t.loc.lon,item_t.loc.lat, 'ro')
        
    # Merchants   
    for item_m in merch_list:
        plt.plot(item_m.loc.lon,item_m.loc.lat, 'bo')
    
    # Submarines
    for item_s in sub_list:
        plt.plot(item_s.loc.lon,item_s.loc.lat, 'go')
        
    # Torpedoes
    for item_p in torp_list:
        plt.plot(item_p.loc.lon,item_p.loc.lat, 'yo') 

    plt.xlim(0,200)
    plt.ylim(0,100)

In [5]:
def Simulator(n_targets,n_merchants,n_submarines,speed_sub,plotter = True,gif = False):
    '''Performs one simulation of a submarine tracking event'''
    
    # Generate enviroment objects
    Targets, Merchants, Submarines = generate_objects(n_merchants,n_targets,n_submarines,speed_sub)
    Torpedoes = []
    
    # Working indexes
    plotter_index = 0
    max_timer = 0
    
    # Clear any existing plots
    plt.clf()
    
    filenames = []
    i = 1
    
    while max_timer < 1e5:
        '''Simulate only as long as submarine is within boundary'''
        # Update postion of eviroment objects 
        target_list = Targets + Merchants
        
        for item_s in Submarines:
            item_s.update_position()
            item_s.ping(target_list)
            if len(item_s.torpedoes) > 0:
                for torpedo in item_s.torpedoes:
                    torpedo.update_position()
                    if torpedo.name not in Torpedoes:
                        Torpedoes.append(torpedo)           

        for item_m in Merchants:
            item_m.update_position()

        for item_t in Targets:
            for item_s in Submarines:
                for item_p in item_s.torpedoes:
                    if item_t.loc.dist_to(item_p.loc) < 1:
                        pass
            item_t.update_position()
            
        plotter_index += 1

        # Plotting only occurs once in 800 steps. Set to improve visual clarity and runtime
        if plotter_index > 1000:
            plotter_index = 0
            if plotter == True:
                contact_picture(Targets,Merchants,Submarines,Torpedoes)
                filename = f'{i}.png'
                i += 1
                filenames.append(filename)
                plt.savefig(filename)
                if gif == True:
                    plt.close()
                
        
        # Ensure if no detections occurs that simulation will halt
        max_timer += 1
    
    if gif == True:
        with imageio.get_writer('mygif.gif', mode='I') as writer:
            for filename in filenames:
                image = imageio.imread(filename)
                writer.append_data(image)

        # Remove files
        for filename in set(filenames):
            os.remove(filename)

    return Targets,Merchants,Submarines

In [ ]:
Targets, Merchants, Submarines = Simulator(30,0,1,16,True,True)

In [ ]:
Submarines[0].tracked